In [1]:
import xml.etree.ElementTree as ET
import vtk

In [4]:
def open_artery_file(filename):
    colors = vtk.vtkNamedColors()
    renderer = vtk.vtkRenderer()

    renWin = vtk.vtkRenderWindow()
    renWin.AddRenderer(renderer)
    iren = vtk.vtkRenderWindowInteractor()
    iren.SetRenderWindow(renWin)

    
    tree = ET.parse(filename)
    root = tree.getroot()  
    graph = root[0]    
    
    # Iterate the nodes and save the positions:
    nodes_vertices = vtk.vtkPoints()
    
    for node in graph.iter('node'):        
        for attribute in node.iter('attr'):            
            if attribute.attrib['name'] == ' position':
                for position_tuple in attribute:
                    node_position = [float(axis_value.text) for axis_value in position_tuple]
                
        nodes_vertices.InsertPoint(int(node.attrib['id'][1:]), node_position)
    
    # Generate a simple unstructured grid with the lines of each edge
    ugrid = vtk.vtkUnstructuredGrid()
    ugrid.Allocate(100)
    for edge in graph.iter('edge'):        
        ugrid.InsertNextCell(vtk.VTK_LINE, 2, [int(edge.attrib['from'][1:]),int(edge.attrib['to'][1:])])
        """
        for attribute in edge.iter('attr'):

            if attribute.attrib['name'] == ' radius':
                for position_tuple in attribute:
                    node_position = [float(axis_value.text) for axis_value in position_tuple]
        """
        
    ugrid.SetPoints(nodes_vertices)
    ugridMapper = vtk.vtkDataSetMapper()
    ugridMapper.SetInputData(ugrid)
    
    ugridActor = vtk.vtkActor()
    ugridActor.SetMapper(ugridMapper)
    ugridActor.GetProperty().SetColor(colors.GetColor3d("Peacock"))
    ugridActor.GetProperty().EdgeVisibilityOn()
    
    renderer.AddActor(ugridActor)
    renderer.SetBackground(colors.GetColor3d("Beige"))
    
    renderer.ResetCamera()
    renderer.GetActiveCamera().Elevation(60.0)
    renderer.GetActiveCamera().Azimuth(30.0)
    renderer.GetActiveCamera().Dolly(1.2)

    renWin.SetSize(640, 480)

    # Interact with the data.
    renWin.Render()

    iren.Start()


In [8]:
filename = 'tree_structure.xml'
open_artery_file(filename)